# Recap Last Week

We covered:
* Pandas basic data structures: Series and Dataframes
* Basics of Data analysis with Pandas
* Data visualization with matplotlib

# This Week

Intermediate Python Topics:

* First class Functions
* Functions within Functions
* Decorators
* Context Managers

## First class Functions

When any object in a programming language is first class, it means that it can be stored in data structures, passed as arguments, or returned from functions.

For example, all of the basic data types in Python (ints, floats, strings, and booleans) are first class. Each of them can be stored, used as arguments, or returned from a function. Because functions are just another kind of object in Python, you can use them in the exact same way.

In [50]:
# define a function and print its type

def first():
    pass

# prints the representation of the function
print(first)

# prints the type of the function
print(type(first))

<function first at 0x000001FCDA8F0620>
<class 'function'>


I've said this before, but Everything in Python is an object, and Python functions are no exception. They are objects who are instances of a 'function' class

**1) In order for an object to be first class it needs to be assignable to variables**

In [7]:
# assign functions to new variables
def second():
    print('second')

b = second

second()
b()

second
second


As you can see we can store the function definition of a function on a new variable, and call it using the new variable name 

**2) In order for an object to be first class it needs to be possible to use it as an argument**

In [51]:
# pass functions into other functions

def takes_a_function(f):
    # call the function f     
    f()
    print('just ran the passed in function')


def hello_world():
    print('Hello World!')
    
takes_a_function(hello_world)

Hello World!
just ran the passed in function


**3) In order for an object to be first class it needs to be something that can be returned from a function**

In [12]:
# return a function from another function

def return_me():
    print('return me')

def return_a_function():
    return return_me

# calling return_a_function returns a function 
return_a_function()

<function __main__.return_me()>

Again, we can see that all functions are instances of a function object, and return_me is no exception

In [52]:
# calling the returned function

return_a_function()()

return me


The above syntax might look weird, and to be honest it is a little weirld if you've never seen it before. What you have to realize is that when you call the function``return_a_function``, it returns a function, which is itself callable, so we can add a new set of ``()`` to call the returned function

**Note: All of the examples so far have used functions that don't have very many arguments (if any at all), just know that you could define these functions to take arguments**

## Functions Within Functions

Just like you can define methods insdie the scope of a class, you can also define functions inside the scope of another function. These locally defined functions will only be accessable from within that function where they are defined

In [19]:
def outside(something):
    # Note that inside is defined within the outside funtion
    def inside(value):
        print(value.__class__)
    # call the inner function from inside
    inside(something)  


outside(3)
outside(3.5)
# not ment to confuse you, but the function can
# take any function, even itself as an argument
outside(outside)

<class 'int'>
<class 'float'>
<class 'function'>


Question: Does ``outisde`` (defined above) return anything?

## Decorators

Decorators are a way to extend the functionality of code without having to rewrite it from the ground up. Decorators provide this functionality by wrapping the code that they want to extend, and they provide this funtionality by utilising the fact that:

    1) functions can be passed as arguments to other functions
    
    2) functions can be defined within other functions
    
    3) functions can be returned from other functions


Decorators are often implemented using functions, but you can also implement them using classes. We'll see examples of both

Decorators can be used to solve a lot of problems quickly, with minimal code adjustments. Any time you want to execute code either before or after some other peice of code is generally a good place to use a decorator. 

A common use for decorators would be to log the amount of time a function takes to run. In many Python web frameworks decorators are used to ensure users are logged in before returning content to them. Really the sky is the limit with decorators.

### Functions as  Decorators

The simplest decorator is a function that takes a function as its only argument, defines an inner function **(with the same call signature as the original function)**, and returns the inner function

    def outer_function(f):
        def wrapper(arg1, arg2, arg3,...):
            # write some code ...
        
        return wrapper

In the following examples we'll define a very basic decorator

In [21]:
def add_1(x):
    return x + 1


def my_decorator(f):
    def wrapper(arg1):
        print('Starting')
        print(f'Result: {f(arg1)}')
        print('Ending', end='\n\n')
    return wrapper

####  Wrap a function on the fly

becuase we can pass funtions into other function we can wrap them any time we want and then store the result into a new variable (or override the existing variable)

In [24]:
# on the fly
wrapped_add_1 = my_decorator(add_1)

wrapped_add_1(2)

Starting
Result: 3
Ending



#### Wrap function when they are defined using @

Using @ followed by the name of the decorator is equivalent to wrapping the function on the fly. The only difference is the syntax is a little nicer.

In [26]:
# use the @ syntax

@my_decorator
def add_2(x):
    return x + 2

add_2(6)

Starting
Result: 8
Ending



### General decorators with \*args and \*\*kwargs

In our previous example we defined the ``my_decorator`` decorator. The only issue with how we defined it, is that its not very flexible. Because the ``wrapper`` function only takes a single argument, it's only able to pass on a single argument to the function. We can generically say that a function takes any number of positional and keyword arguments using \*args and \*\*kwargs

**NOTE: You don't need to refer to these varibales as args and kwargs, but thats what they are typically called**

To illustrate the point, I'll define a funtion that takes two arguments and try to wrap it with our first decorator. I'll try to call the function using both arguments, but unfortunately we'll get an error because more arguments are provided than the function expects

In [27]:
@my_decorator
def take_two(a, b):
    print(a, b)

take_two(1, 2)

TypeError: wrapper() takes 1 positional argument but 2 were given

**NOTE: The error function is actually pretty interesting. Above, we called take_two yet the error message is saying that wrapper was the function that was called with two arguments. When working with decorated functions, you might think you're calling the original function, but in reality you're calling whatever funtion was returned by the decorator.**

**Decorting a function using the @ syntax is equivalent to wrapping some new piece of code around an existing funtion, and then reassigning the name of the function to the original name**

In [41]:
import time

# a more general decorator
# The outer funtion still takes a function as an argument
def timer(f):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        total_ms = end - start
        print(f'It took {total_ms} seconds to run {f.__name__}')
        return result
    return wrapper

@timer
def loop_over_items(iterable, sleep_time=.1):
    for item in iterable:
        time.sleep(sleep_time)

@timer
def square(value):
    return value ** 2

loop_over_items('Hey')
loop_over_items(range(5))

square(2000)
square(20)

It took 0.31611061096191406 seconds to run loop_over_items
It took 0.5470714569091797 seconds to run loop_over_items
It took 0.0 seconds to run square
It took 0.0 seconds to run square


400

### Stacking decorators

In most cases you can keep wrapping decorators as often as you need. The fist decorator defined on a function will be the last one to run

In [59]:
def count_called(f):
    count = 0
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'{f.__name__} ran {count} time(s)')
        return f(*args, **kwargs)
    return wrapper

@timer
@count_called
def cube(x):
    return x ** 3

for num in range(10):
    print(cube(num))    

cube ran 1 time(s)
It took 0.0 seconds to run wrapper
0
cube ran 2 time(s)
It took 0.0 seconds to run wrapper
1
cube ran 3 time(s)
It took 0.0 seconds to run wrapper
8
cube ran 4 time(s)
It took 0.0 seconds to run wrapper
27
cube ran 5 time(s)
It took 0.0 seconds to run wrapper
64
cube ran 6 time(s)
It took 0.0009989738464355469 seconds to run wrapper
125
cube ran 7 time(s)
It took 0.0 seconds to run wrapper
216
cube ran 8 time(s)
It took 0.0 seconds to run wrapper
343
cube ran 9 time(s)
It took 0.0 seconds to run wrapper
512
cube ran 10 time(s)
It took 0.0 seconds to run wrapper
729


### Classes as Decorators

creating a decorator out of a class takes advantage of the ``__call__`` dunder method on classes. This allows a class to become callable

In [48]:
class LogArguments:
    def __call__(self, f):
        def wrapper(*args, **kwargs):
            print(args)
            print(kwargs)
            return f(*args, **kwargs)
        return wrapper

@LogArguments()
def my_sum(*args):
    return sum(args)


my_sum(*range(10))

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
{}


45

## Context Managers

Context managers are objects that handle setup and teardown of resources for you. Becuase the setup and teardown is handled in the context manager the code you write will be cleaner and easier to read. The benefit of using a context manager to handle setup for you is that resources will always be cleaned up regardless of if an error occues in the context or not.

### ``__enter__`` and ``__exit__``

To use an object as a context manager you use Python's ``with`` statement. In order for an object to be treated as a context manager it needs to define both an ``__enter__`` and an ``__exit__`` method like so:

    
    def __enter__(self):
        # write your code ...
       
    def __exit__(self, exception, value, traceback):
        # write your code ...

Note that ``__exit__`` will be passed information about exceptions that may occur while inside the context of the context manager. If an exception occurs and the ``__exit__`` method returns False, the exception will be thrown, otherwise the exception will be supressed 

In the following example, we'll define a temporary file context manager.

**NOTE: This functionality is built into the standard library using the tempfile module**

In [72]:
import os
import pathlib

class TempFile:
    def __init__(self, file_path):
        self.path = pathlib.Path(file_path)
    
    def __enter__(self):
        self.file = open(self.path, mode='w')
        return self.file
    
    def __exit__(self, exception, value, traceback):
        if exception:
            print('oops, and error occured!')
        print('closing the file')
        self.file.close()
        os.remove(self.path)
        return True
            
    
    
def does_file_exist(file_path):
    if os.path.exists(file_path):
        print(f'Yes, {file_path} exists!')
    else:
        print(f'No, {file_path} does not exist!')
    
example_file1 = 'newfile1.txt'
    
with TempFile(example_file1) as f:
    user_input = input('Check that the new file was created. Press Enter to continue')
    does_file_exist(example_file1)

# For some reason you need to save for the file menu
# on the side to update
does_file_exist(example_file1)  

example_file2 = 'newfile2.txt'

with TempFile(example_file2) as f:
    does_file_exist(example_file2)
    raise Exception

does_file_exist(example_file2)

Check that the new file was created. Press Enter to continue 


Yes, newfile1.txt exists!
closing the file
No, newfile1.txt does not exist!
Yes, newfile2.txt exists!
oops, and error occured!
closing the file
No, newfile2.txt does not exist!


### Contextmanager Decorator

If you don't want to define context manager classes, Python provides some tools in the standard library for you to be able to define context managers using functions. In order to do so, you'll need to use the ``contextmanager`` decorator provided to you by the built in ``contextlib`` module

You can define your context manager as a generator, yielding the result and handling any cleanup and teardown with ``try, except, else, and finally`` blocks

In [60]:
from contextlib import contextmanager

# timer context manager
@contextmanager
def timer_contex():
    try:
        start = time.time()
        # allow code inside the context to run         
        yield
    finally:
        end = time.time()
        print(end - start)
        

        
with timer_contex():
    for x in range(6):
        time.sleep(.1)   

0.6316969394683838


## Additional Resources

* [Python Inner Functions (Blog Post)](https://realpython.com/inner-functions-what-are-they-good-for/)
* [Python Decorators (Blog Post)](https://realpython.com/primer-on-python-decorators/)
* [Python Decorators (YouTube Video)](https://www.youtube.com/watch?v=FsAPt_9Bf3U&t=25s)
* [First Class Functions (YouTube Video)](https://www.youtube.com/watch?v=kr0mpwqttM0)
* [Closures (YouTube Video)](https://www.youtube.com/watch?v=swU3c34d2NQ)
* [Context Manager Use Cases (Blog Post)](http://arnavk.com/posts/python-context-managers/) The artilce might be a little hard to fully understand, but the example use cases are pretty diverse